In [132]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError

# Quantidade de notas fiscais
numero_inicial_nota = 1104
numero_maximo_nota = 1145
quant_maxima_itens = 10

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)

lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = fake.date_time_between(start_date=datetime(2024,1,1),end_date=datetime(2024,1,20))
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

# Caminhos das pastas
pastas = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}
# Dicionário com os DataFrames e suas pastas correspondentes
dataframes = {
    'ITEMNOTA': df_ITEMNOTA,
    'NOTAFISCAL': df_NOTAFISCAL,
    'VENDA': df_VENDA
}

# Salvar cada DataFrame em um arquivo CSV na pasta correspondente
for nome_df, df in dataframes.items():
    try:
        id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
        # Nome do arquivo
        nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
        
        # Caminho completo do arquivo
        caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
        
        # Salvar o DataFrame em um arquivo CSV
        df.to_excel(caminho_completo, index=False)
        print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
    except Exception as e:
        print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
DataFrame ITEMNOTA salvo com sucesso em: C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA\arquivo_ITEMNOTA_vendas_1104ate_1145.xlsx
DataFrame NOTAFISCAL salvo com sucesso em: C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL\arquivo_NOTAFISCAL_vendas_1104ate_1145.xlsx
DataFrame VENDA salvo com sucesso em: C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA\arquivo_VENDA_vendas_1104ate_1145.xlsx


In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []
    

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))
            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)

lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1003
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = fake.date_time_between(start_date=datetime(2024,1,1),end_date=datetime(2024,1,20))
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

#display(df_ITEMNOTA)
display(df_NOTAFISCAL)
#display(df_VENDA)

tipo_dados_ITEMNOTA = {
                'ID_NOTAFISCAL':  INTEGER,
                'ID_PRODUTO':     INTEGER,
                'QUANTIDADE':     INTEGER,
                'PRECO_UNIT':     NUMERIC(18,2),
                'TOTAL_ITEM':     NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_NOTAFISCAL = {
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_VENDA = {
                'DATA':           DATE,
                'ID_NOTAFISCAL':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_FORMA_PGMT':  INTEGER,
                'TOTAL_VENDA':    NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}
try:
    with engine.connect() as conn:
            conn.execute(text(f"TRUNCATE TABLE ITEMNOTA"))  #limpa os dados existentes, pode ser removido do codigo
            conn.commit()

    df_ITEMNOTA.to_sql(name='ITEMNOTA',con=engine,if_exists='append',index=False,dtype=tipo_dados_ITEMNOTA)

    engine.dispose()

    with engine.connect() as conn:
            conn.execute(text(f"TRUNCATE TABLE NOTAFISCAL"))  #limpa os dados existentes, pode ser removido do codigo
            conn.commit()

    df_NOTAFISCAL.to_sql(name='NOTAFISCAL',con=engine,if_exists='append',index=False,dtype=tipo_dados_NOTAFISCAL)

    engine.dispose()

    with engine.connect() as conn:
            conn.execute(text(f"TRUNCATE TABLE VENDA"))  #limpa os dados existentes, pode ser removido do codigo
            conn.commit()

    df_VENDA.to_sql(name='VENDA',con=engine,if_exists='append',index=False,dtype=tipo_dados_VENDA)
    
    print("Dados inseridos com sucesso.")
except SQLAlchemyError as e:
     print(f"Erro ao inserir dados na tabela: {e}")
engine.dispose()


In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
import numpy as np

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []
    

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))
            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)

lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1010
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = fake.date_time_between(start_date=datetime(2024,1,1),end_date=datetime(2024,1,20))
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)

valores_unicos = df_todas_notas["CHAVE_ACESSO"].unique()

valores_numero_nota = df_todas_notas["Numero_Nota"]
valores_unicos_id_cfop = df_todas_notas["ID_CFOP"]

#Transformar a coluna ID_CFOP de varios valores para o mesmo Numero de nota para valores UNICOS por Numero nota
primeiros_numeros_notas = []
indice = 0

def agrupar_sequencias_numero_notas(numero_nota):
    sequencias = []
    if len(numero_nota) == 0:
        return sequencias
    
    numero_nota_atual = numero_nota[0]
    inicio = 0
    
    for i, num_nota in enumerate(numero_nota):
        if num_nota != numero_nota_atual:
            sequencias.append((numero_nota_atual, inicio, i - 1))  # (nome, início, fim)
            numero_nota_atual = num_nota
            inicio = i
    sequencias.append((numero_nota_atual, inicio, len(numero_nota) - 1))  # Última sequência
    return sequencias

sequencias_numero_notas = agrupar_sequencias_numero_notas(valores_numero_nota.tolist())
print('sequencia_numero_notas')
display(sequencias_numero_notas)

for num_nota, inicio, fim in sequencias_numero_notas:
    # Adicionar a primeira idade da sequência correspondente ao nome
    primeiros_numeros_notas.append(valores_unicos_id_cfop[indice])
    
    # Avançar o índice para o início da próxima sequência de idades
    indice += (fim - inicio + 1)

df_id_cfop = pd.DataFrame({
                    "NUMERO_NOTA":   [num_nota for num_nota, _, _ in sequencias_numero_notas],
                    "ID_CFOP":       primeiros_numeros_notas, 

})

df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_id_cfop['ID_CFOP'],
                'CHAVE_ACESSO':   valores_unicos,
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':    df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

display(df_ITEMNOTA)
display(df_NOTAFISCAL)
#display(df_VENDA)

tipo_dados_ITEMNOTA = {
                'ID_NOTAFISCAL':  INTEGER,
                'ID_PRODUTO':     INTEGER,
                'QUANTIDADE':     INTEGER,
                'PRECO_UNIT':     NUMERIC(18,2),
                'TOTAL_ITEM':     NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_NOTAFISCAL = {
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

tipo_dados_VENDA = {
                'DATA':           DATE,
                'ID_NOTAFISCAL':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_FORMA_PGMT':  INTEGER,
                'TOTAL_VENDA':    NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
}

#with engine.connect() as conn:
#        conn.execute(text(f"TRUNCATE TABLE ITEMNOTA"))  #limpa os dados existentes, pode ser removido do codigo
#        conn.commit()
#
#df_ITEMNOTA.to_sql(name='ITEMNOTA',con=engine,if_exists='append',index=False,dtype=tipo_dados_ITEMNOTA)
#
#engine.dispose()
#
#with engine.connect() as conn:
#        conn.execute(text(f"TRUNCATE TABLE NOTAFISCAL"))  #limpa os dados existentes, pode ser removido do codigo
#        conn.commit()
#
#df_NOTAFISCAL.to_sql(name='NOTAFISCAL',con=engine,if_exists='append',index=False,dtype=tipo_dados_NOTAFISCAL)
#
#engine.dispose()
#
#with engine.connect() as conn:
#        conn.execute(text(f"TRUNCATE TABLE VENDA"))  #limpa os dados existentes, pode ser removido do codigo
#        conn.commit()
#
#df_VENDA.to_sql(name='VENDA',con=engine,if_exists='append',index=False,dtype=tipo_dados_VENDA)

engine.dispose()
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped
    
tabela = "ITEMNOTA"
coluna = "ID_NOTAFISCAL"
tabela2 = "NOTAFISCAL"
coluna2= "CHAVE_ACESSO"
tabela3 = "VENDA"
coluna3 = "?"


with engine.connect() as conn:
    # Consulta os valores existentes na coluna "ID_NOTAFISCAL" da tabela "ITEMNOTA"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
    query2 = text(f"SELECT {coluna2} FROM {tabela2}")
    lista_valores_existentes2 = pd.read_sql(query2, conn)[coluna2].tolist()


    # Filtra os valores que não existem na tabela
    df_produto_filtrado = df_ITEMNOTA[~df_ITEMNOTA["ID_NOTAFISCAL"].isin(lista_valores_existentes)]
    df_produto_filtrado2 = df_NOTAFISCAL[~df_NOTAFISCAL["CHAVE_ACESSO"].isin(lista_valores_existentes2)]
    print('df_produto_filtrado')
    display(df_produto_filtrado)
    print('df_produto_filtrado2')
    display(df_produto_filtrado2)
    # Se houver valores para adicionar
    if not df_produto_filtrado.empty or not df_produto_filtrado2.empty:
        # Limpa os espaços desnecessários
        df_produto_filtrado = df_produto_filtrado.apply(strip_all_types)
        df_produto_filtrado2 = df_produto_filtrado2.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_ITEMNOTA = {
                'ID_NOTAFISCAL':  INTEGER,
                'ID_PRODUTO':     INTEGER,
                'QUANTIDADE':     INTEGER,
                'PRECO_UNIT':     NUMERIC(18,2),
                'TOTAL_ITEM':     NUMERIC(18,2),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
                }

        tipo_dados_NOTAFISCAL = {
                'DATA':           DATE,
                'TOTAL_NOTA':     NUMERIC(18,2),
                'ID_FORMA_PGMT':  INTEGER,
                'ID_CLIENTE':     INTEGER,
                'ID_LOJA':        INTEGER,
                'ID_CFOP':        INTEGER,
                'CHAVE_ACESSO':   VARCHAR(100),
                'DATA_CRIACAO':   DATE,
                'DATA_ATLZ':      DATE,
                'VERSAO':         INTEGER,
                }

        # Exibe os dados que serão adicionados
        print("Dados a serem adicionados:")
        display(df_produto_filtrado)
        print("Dados a serem adicionados NOTA FISCAL:")
        display(df_produto_filtrado2)

        # Adiciona os dados ao banco de dados
        df_produto_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_ITEMNOTA
        )

        df_produto_filtrado2.to_sql(
            name=tabela2,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_NOTAFISCAL
        )

        print("Dados inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

# Fecha a conexão com o banco de dados
engine.dispose()

In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

#GERAR ARQUIVOS .CSV

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"


conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar data e hora aleatória
def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  # Meses com 30 dias
        dia = random.randint(1, 30)
    elif mes == 2:  # Fevereiro
        dia = random.randint(1, 28)
    else:  # Meses com 31 dias
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    return data_completa

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []
    

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        print(produto)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, data_atlz, versao,id_produto))
            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
display(df_prod)

lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

#print(lista_IDPRODUTO)
#print(len(lista_IDPRODUTO))

## Exemplo de DataFrame de produtos e preços unitários
#data = {
#    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
#    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
#}
#df_prod_valores = pd.DataFrame(data)

#rewmover
#listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"]

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1008
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []


# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024, 2024)
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)
   


# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)


df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

#Exibir todas as notas fiscais
#print("\nTodas as Notas Fiscais:")
#print(df_todas_notas)

display(df_ITEMNOTA)
#display(df_NOTAFISCAL)
#display(df_VENDA)
#

In [ ]:
#Script para gerar Tabela Item Nota

import random
import pandas as pd

# Exemplo de DataFrame de produtos e preços unitários
data = {
    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
}
df_prod_valores = pd.DataFrame(data)

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens):
    # Gerar quantidade aleatória de itens únicos na nota fiscal
    quant_item_unicos = random.randint(1, quant_maxima_itens)
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")
    
    lista_saida = [] # Lista para armazenar os itens da nota fiscal

    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((numero_nota, produto, quantidade, preco_unit, preco_total_item))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[4] for item in lista_saida), 2)

    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida,
        columns=["Número da Nota", "Produto", "Quantidade", "Preço Unitário", "Total do Item", "Total da Nota"]
    )

    return df_nota_fiscal

# Lista de produtos
listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle", "Pepsi", "Guarana Antarctica"]

# Gerar uma nota fiscal
numero_nota = 1000  # Número da nota fiscal
quant_maxima_itens = 10
df_nota_fiscal = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens)

# Exibir a nota fiscal
print("\nNota Fiscal:")
print(df_nota_fiscal)
print(f"\nTotal da Nota: R$ {df_nota_fiscal['Total da Nota'].iloc[0]:.2f}")

In [ ]:
#Script para gerar Tabela de Notas Fiscais

import random
import pandas as pd
from datetime import datetime

# Exemplo de DataFrame de produtos e preços unitários
data = {
    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
    
}
df_prod_valores = pd.DataFrame(data)

def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  
        dia = random.randint(1, 30)
    elif mes == 2:  
        dia = random.randint(1, 28)
    else:  
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    
    return data_completa

#Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    
    quant_item_unicos = random.randint(1, quant_maxima_itens) #Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")
    
    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    
    id_forma_pgmt = random.randint(1,8)
    id_cliente = random.randint(1,1000)
    id_loja = random.randint(1,150)
    id_cfop = random.randint(1,5)

    #Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)              # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)                    # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            data = data_nota
            id_forma_pgmt = id_forma_pgmt
            id_cliente = id_cliente
            id_loja = id_loja
            id_cfop = id_cfop
            lista_saida.append((data, numero_nota, produto, quantidade, preco_unit, preco_total_item, id_forma_pgmt, id_cliente, id_loja, id_cfop))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)

    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]

    #Data da nota fiscal
    #data_nota = gerar_data_hora(2024,2024)


    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida,
        columns=["Data","Número da Nota", "Produto", "Quantidade", "Preço Unitário", "Total do Item","Id Forma Pgmt","Id Cliente","Id Loja","Id CFOP", "Total da Nota"]
    )

    return df_nota_fiscal

# Lista de produtos
listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle", "Pepsi", "Guarana Antarctica"]

#Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 1005  
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024,2024)
    df_nota_fiscal = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)

# Exibir todas as notas fiscais
print("\nTodas as Notas Fiscais:")
print(df_todas_notas)

# Exibir o total de cada nota fiscal
#print("\nResumo dos Totais das Notas Fiscais:")
#for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
#    total_nota = df_todas_notas.loc[df_todas_notas["Número da Nota"] == numero_nota, "Total da Nota"].iloc[0]
#    print(f"Nota {numero_nota}: R$ {total_nota:.2f}")